In [1]:
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DistilBertForSequenceClassification, DistilBertTokenizer,
import torch
from evaluate import evaluate
from warp_training import train_warp

In [2]:
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

Проведем эксперименты с изменением T

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
sft_model = GPT2LMHeadModel.from_pretrained('lvwerra/gpt2-imdb').to(device)
reward_model = DistilBertForSequenceClassification.from_pretrained('results/reward/checkpoint-4221').to(device)
reward_tokenizer = DistilBertTokenizer.from_pretrained('results/reward/checkpoint-4221')
sft_model_tokenizer = GPT2Tokenizer.from_pretrained('lvwerra/gpt2-imdb')
sft_model_tokenizer.pad_token = sft_model_tokenizer.eos_token

In [7]:
reward_warp = {}
kl_div_dict = {}
for T in [100, 150, 200]:
    # Обучение модели с заданными параметрами
    config['training_params']['iterations'] = T
    warp_model = train_warp(config)
    
    # Оценка моделей
    warp_rewards, sft_rewards, kl_div = evaluate(
        sft_model,
        warp_model,
        sft_model_tokenizer,
        reward_model,
        reward_tokenizer,
        device
    )
    
    # Сохранение результатов
    reward_warp[T] = warp_rewards
    kl_div_dict[T] = kl_div

# Вывод результатов
print('Reward Warp:', reward_warp)
print('KL Divergence Dict:', kl_div_dict)

iteration: 0 of 200:   0%|          | 0/100 [00:00<?, ?it/s]c:\ProgramData\anaconda3\envs\loop_tf\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\konstantin\Desktop\alignment\warp_training.py:104: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1581.)
  ema_param.data.mul_(1 - mu).add_(mu, param.data)
iteration: 4 of 200:  43%|████▎     | 43/100 [02:37<03:33,  3.74s/it]

По графику видим, что при увеличении итераций reward и KL увеличивается, т.к. мы напрямую оптимизируем нашу модель под ревард, а KL будет расти, потому что мы отдаляемся от изначального распределения, хотя мы и пытаемся оставлять ее на месте.
Хочется подвести итоги по результатам экспериментов в ходе реализации алгоритма, о которых я не успел подготовить наглядные примеры. При увеличении $mu$ в EMA наша модель будет все дальше отходить от нашей изначальной модели в сторону policy, что увеличит KL и reward и мы берем ее достаточно маленькой, чтобы не терять начальное распределение и не терять обобщающую способность модели.
$lambda$ определяет насколько модель будет склонна к одной из двух (или больше) моделей, которые мы интерполируем, поэтому нельзя однозначно сказать тенденция к росту или падению метрик.
Аналогично при увеличении $thetta$ мы все больше отдаляемся от изначальных параметров модели в процессе интерполяции, что увеличит KL. Можно думать об этом как шаге в градиентном спуске (мы идем от начальной модели к желаемой с lr = $thetta$).

Не успел: попробовать scheduler для $thetta$, оценить зависимость качества моделей от батчей и количества подаваемых данных
